In [2]:
import pwv_kpno 
major, minor, patch = pwv_kpno.__version__.split('.')

if not (int(minor) > 12 or (int(minor) == 12 and int(patch) >= 1)):
    raise ImportError('This demo requires at least pwv_kpno version 0.12.1')


In [25]:
import numpy as np
import matplotlib.pyplot as plt

from pwv_kpno.pwv_atm import trans_for_pwv

bins = np.arange(4000, 12000)
t = trans_for_pwv(12, 4, bins)

plt.errorbar(x=t['wavelength'], y=t['transmission'], yerr=t['transmission_err'], ecolor='grey')
plt.ylim(0, 1)
plt.show()

t

ValueError: Arguments "names" and "dtype" must match number of columns

## Creating a new config file

In order to model the atmosphere at a given location, `pwv_kpno` requires a config file for that location. We can create a custom config file using the `ConfigBuilder` class as follows:

In [2]:
import numpy as np
from pwv_kpno import ConfigBuilder


# An subset of the Kitt Peak model using Angstroms and cm^2
wavelengths = np.array([3000, 3001, 3002]) 
modtran_cross_sections = np.array([5.719913264564206E-9,
                                   5.725763008721962E-9,
                                   5.7315124715513006E-9])

new_config = ConfigBuilder()
new_config.site_name = 'example_loc'
new_config.primary_rec = 'KITT' # The primary receiver
new_config.sup_rec = ['AZAM']  # A list of secondary receivers
new_config.wavelengths = wavelengths
new_config.cross_sections = modtran_cross_sections

# Create output file example_loc.ecsv
# By default existing files are not overwritten
new_config.save_to_dir('.', overwrite=True)


An alternative but functionally equivilent method would be to instantiate the class with the desired information:

In [4]:
new_config = ConfigBuilder(
    site_name = 'example_loc',
    primary_rec = 'KITT',
    sup_rec = ['AZAM'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections
)

new_config.save_to_dir('.', overwrite=True)

If desired, useres can specify custom data cuts on SuomiNet data used by the package. Data cuts can be specified using a 2d dictionary of boundary values. The first key specifies which receiver the data cuts apply to. The second key specifies what values to cut. Following SuomiNet's naming convention, optional values that can be cut include `'PWV', 'PWVerr', 'ZenithDelay', 'SrfcPress', 'SrfcTemp',` and `'SrfcRH'`. For example, if we only wanted to use pressure measurements between 880 and 925 mbar for the AZAM receiver:

In [6]:
# Here we define what data NOT to use
data_cuts = {'AZAM': {'SrfcPress': [[0, 880], [925, 10000]]}}     

new_config = ConfigBuilder(
    site_name = 'example_loc',
    primary_rec = 'KITT',
    sup_rec = ['AZAM'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections,
    data_cuts = data_cuts
)

new_config.save_to_dir('.', overwrite=True)

## Errors and Warnings

If the user forgets to provide any required information, `save_to_dir` will raise an error.

In [7]:
new_config = ConfigBuilder(
    primary_rec = 'kitt',
    sup_rec = ['Sa48'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections
)

new_config.save_to_dir('.', overwrite=True)

ValueError: Must specify attribute site_name before saving.

Any issues with formatting the data will raise warnings. This helps ensure that the user understands how the package works. These warnings are also raised when the `save_to_dir` method is called.

In [9]:
new_config = ConfigBuilder(
    site_name = 'EXAMPLE_LOC',
    primary_rec = 'kitt',
    sup_rec = ['Sa48'],
    wavelengths = wavelengths,
    cross_sections = modtran_cross_sections
)

new_config.save_to_dir('.', overwrite=True)

/Users/daniel/GitHub/pwv_kpno/pwv_kpno/_config_builder.py:107: UserWarning: pwv_kpno uses lowercase site names. The gicen site name EXAMPLE_LOC will be saved as example_loc.
  ' be saved as {}.'.format(self.site_name, self.site_name.lower())
/Users/daniel/GitHub/pwv_kpno/pwv_kpno/_config_builder.py:127: UserWarning: SuomiNet ID codes should be uppercase. ID code Sa48 will be saved as SA48.
  ' be saved as {}.'.format(id_code, id_code.upper())
/Users/daniel/GitHub/pwv_kpno/pwv_kpno/_config_builder.py:127: UserWarning: SuomiNet ID codes should be uppercase. ID code kitt will be saved as KITT.
  ' be saved as {}.'.format(id_code, id_code.upper())


# Choosing a location into pwv_kpno

When imported, the `pwv_kpno` package has no knowledge of what location you wish to model

In [10]:
from pwv_kpno import pwv_atm

pwv_atm.available_data()

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

We can configure `pwv_kpno` to model a specific location using the `settings` object

In [12]:
from pwv_kpno import settings

settings.set_site('kitt_peak')
pwv_atm.available_data()

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]

The `settings` object also provides a handful of useful information about the current location

In [15]:
print('Location being modeled:', settings.site_name)
print('List of available locations: ', settings.available_sites)
print('List of receiver ids for the current location: ', settings.receivers)
print('The id for primary receiver: ', settings.primary_rec)
print('The ids for supplementary receivers: ', settings.supplement_rec)

Location being modeled: kitt_peak
List of available locations:  ['kitt_peak']
List of receiver ids for the current location:  ['AZAM', 'KITT', 'P014', 'SA46', 'SA48']
The id for primary receiver:  KITT
The ids for supplementary receivers:  ['AZAM', 'P014', 'SA46', 'SA48']


In order to model a custom location we first need to import the appropriate config file. Note that this step only has to be done once. Afterwards we can switch betwean locations normally using the `set_location` method.

In [20]:
settings.import_site('./example_loc.ecsv', overwrite=True)
settings.set_site('example_loc')

print('Location being modeled:', settings.site_name)
print('List of available locations: ', settings.available_sites)
print('List of receiver ids for the current location: ', settings.receivers)
print('The id for primary receiver: ', settings.primary_rec)
print('The ids for supplementary receivers: ', settings.supplement_rec)


Location being modeled: example_loc
List of available locations:  ['kitt_peak', 'example_loc']
List of receiver ids for the current location:  ['KITT', 'SA48']
The id for primary receiver:  KITT
The ids for supplementary receivers:  ['SA48']


By default there is no local SuomiNet data for this location. We need to download this first

In [21]:
pwv_atm.update_models(2010)

/Users/daniel/anaconda/lib/python3.6/site-packages/scipy/odr/odrpack.py:856: OdrWarning: Empty data detected for ODR instance. Do not expect any fitting to occur
  OdrWarning)


[2010]

In [22]:
pwv_atm.measured_pwv()

date,KITT,KITT_err,SA48,SA48_err
UTC,mm,mm,mm,mm
object,int64,int64,float64,float64
2010-01-01 00:15:00+00:00,--,--,4.7,0.125
2010-01-01 00:45:00+00:00,--,--,4.7,0.125
2010-01-01 01:15:00+00:00,--,--,3.6,0.125
2010-01-01 01:45:00+00:00,--,--,3.6,0.125
2010-01-01 02:15:00+00:00,--,--,3.5,0.125
2010-01-01 02:45:00+00:00,--,--,3.6,0.125
2010-01-01 03:15:00+00:00,--,--,3.7,0.125
2010-01-01 03:45:00+00:00,--,--,3.5,0.125
